In [1]:
import pandas as pd  
pd.options.mode.chained_assignment = None
import numpy as np  
from copy import deepcopy
from string import punctuation
from random import shuffle
from sklearn.preprocessing import scale
import advertools as adv


#import gensim
#from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
#LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

In [12]:
#data = pd.read_csv("final_df.csv", encoding='utf-8')
#data.iloc[:,1:].to_csv("final_df_.csv", index = False)
data = pd.read_csv("df_vacunacion.csv", encoding='utf-8')

In [13]:
def remove_character(tweet, characters):
    #characters = "'!?"
    tweet = ''.join( x for x in tweet if x not in characters)
    return tweet

def extract_emojis(str): 
    return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)

In [14]:
#text_list = "🤔 🙈 me así, bla es se 😌 ds 💕👭👙"
text_list  =  "🇵🇪 ADELANTO 📺 | «[@KeikoFujimori] está demostrando al #Perú entero y al mundo que no sabe perder y no sabe aceptar su derrota"
emoji_summary = adv.extract_emoji(text_list)
print(emoji_summary['emoji_flat'])

['📺']


In [15]:
#function to retunr specific persons
def clean_tweet(df, label_tweet, characters):
    df['tweet_'] = df.apply(lambda x: remove_character(x[label_tweet], characters), axis=1)
    return df

#A = clean_tweet(data, 'tweet', '@#')
#A[['tweet', 'tweet_']]
def extract_components(tweet):
    subject = []
    emoticons = []
    tweet  = str(tweet).lower()
    #tweet = remove_character(tweet,'@#')
    tokens = tokenizer.tokenize(tweet)
    subjects = list(filter(lambda t: t.startswith('@'), tokens))
    
    emoji = adv.extract_emoji(tweet)
    emoticons = emoji['emoji_flat']
    #print(subjects, emoticons)
    return subjects, emoticons
    
def extract_labels(df, label_tweet):
    #print(df.apply(lambda x: print(extract_components(x[label_tweet])), axis=1))
    df['subject'] = df.apply(lambda x: extract_components(x[label_tweet])[0], axis=1)
    df['emoticons'] = df.apply(lambda x: extract_components(x[label_tweet])[1], axis=1)
    return df
          
df_tweets = extract_labels(data, 'tweet')

In [16]:
#seleccionando las columnas mas importantes
df_tweets = df_tweets[['id', 'date', 'tweet', 'language', 'hashtags', 'user_id', 'username', 'name', 'retweet', 'nlikes', 'nreplies', 'subject', 'emoticons']]

In [17]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def remove_usernames_links(tweet):
    #tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub(r"http\S+", "", tweet)
    tweet = remove_emojis(tweet)
    #print(tweet)
    return tweet
#df['tweet'] = df['tweet'].apply(remove_usernames_links)

In [18]:
def pre_clean_tweets(df, label):
    #print(df)
    df['tweets_clean'] = df.apply(lambda x: remove_usernames_links(x[label]), axis=1)
    return df
    #df['tweets_clean'] = df.apply(lambda x: strip_all_entities(x[label]), axis=1)
    
df_tweets = pre_clean_tweets(df_tweets,'tweet')
#df_tweets

In [19]:
#df_tweets.to_csv('final_df_elecciones.csv', index = False)
df_tweets.to_csv('final_df_vacunacion.csv', index = False)

In [20]:
len(df_tweets)

1343

In [11]:
df_tweets

,id,date,tweet,language,hashtags,user_id,username,name,retweet,nlikes,nreplies,subject,emoticons,tweets_clean
0,1508817563714740232,2022-03-29 11:45:14,"@CIDH @CorteIDH #Peru En investigaciones, el n...",es,"['peru', 'fujimorismonuncamas', 'keikonova']",780993618,KMartinBF,Martín Baldárrago Fernández,False,0,0,"[@cidh, @corteidh]",[],"@CIDH @CorteIDH #Peru En investigaciones, el n..."
1,1506255176885325826,2022-03-22 10:03:13,"@GuilleBermejo01 el ex-presidente Sagasti es, ...",es,['keikonova'],1526274745,sobrime,JULIO SOBRINO,False,0,0,[@guillebermejo01],[],"@GuilleBermejo01 el ex-presidente Sagasti es, ..."
2,1504796037696958469,2022-03-18 09:25:07,Perú siempre peleando la misma batalla contra ...,es,"['fujimorigenocida', 'fujimorinuncamas', 'keik...",1402317169262837761,coca_cori,Chañan Cori Coca,False,0,0,[],[],Perú siempre peleando la misma batalla contra ...
3,1496245966952976385,2022-02-22 19:10:12,Vote x Castillo en la segunda vuelta ante la a...,es,"['narcoestado', 'keikonova']",705421458021994496,AlisParraB,Alicia Parra,False,0,0,[],[],Vote x Castillo en la segunda vuelta ante la a...
4,1481936501327998977,2022-01-14 07:29:29,@willaxtv Ese es el pensamiento fujimorista y ...,es,"['fujimorismonuncamas', 'noalamafiafujimorista...",1120023349504167937,Jucevi2,Jucevi,False,0,0,"[@willaxtv, @keikofujimori]",[],@willaxtv Ese es el pensamiento fujimorista y ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,1460385159426019329,2021-11-15 20:12:09,Castillo gobierna el Perú con sus amigazos de ...,es,['vacanciapredrocastilloya'],730799927593447425,ValeriaBersaken,Valeria Bersaken,False,0,0,[],[],Castillo gobierna el Perú con sus amigazos de ...
1609,1460353340068483074,2021-11-15 18:05:42,Perú Libre escenifica una lucha interna fictic...,es,"['vacanciapresidencialya', 'vacanciapredrocast...",946239797798023168,MiguelTolenti1,Miguel Angel Tolentino Espinoza,False,0,0,[],[],Perú Libre escenifica una lucha interna fictic...
1610,1460301787693568008,2021-11-15 14:40:51,@SUTEP_Peru @gabrielalpp ¿Qué esperan que no s...,es,['vacanciapredrocastilloya'],1407102833728577541,Libertarios15,LibertarioConservador,False,2,1,"[@sutep_peru, @gabrielalpp]",[],@SUTEP_Peru @gabrielalpp ¿Qué esperan que no s...
1611,1460298705085546501,2021-11-15 14:28:36,Eres o no!! lo que queremos es que se vayan to...,es,"['vacanciapresidencialya', 'vacanciapredrocast...",1451566003654168590,Veroquis15,Veronica Quispe,False,1,0,[],[],Eres o no!! lo que queremos es que se vayan to...
